In [1]:
#Importo librerías necesarias
import numpy as np
from numpy.linalg import norm
#import matplotlib.pyplot as plt
#import matplotlib.cm as cm
#%matplotlib inline
#import os
from visual import *

/usr/lib/python2.7/dist-packages/visual/materials.py:70: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.__setattr__(key, value)


In [2]:
#Definición de la constante de gravitación universal
G = 2.824381e-7

In [3]:
def f(Y,M):
    global G
    info = [] #Toda la información se guarda en esta variable local, que al final del método es retornada.
    for i in range(0,len(Y),2): #Este ciclo recorre cada una de las masas.
        v = 0
        #Este ciclo encuentra la atracción que siente cada una de las masas efectuada por las demás.
        for j in range(0,len(Y),2):
            if (i != j): #Utilizamos esta condición para evitar calcular la fuerza que una masa ejerce sobre sí misma.
                v -= G*(M[int(j/2)])*((Y[i] - Y[j]) / norm(Y[i] - Y[j])**3)
        info.extend((Y[i+1],v))
    informacion = np.array(info)
    return(informacion)

In [4]:
def runge_kutta_orden4(dt,Y,M):
    #Resolvemos las ecuaciones de movimiento con Runge-Kutta orden 4.
    K1 = f(Y,M)
    K2 = f(Y + dt*(1./2.*K1),M)
    K3 = f(Y + dt*(1./2.*K2),M)
    K4 = f(Y + dt*K3,M)
    #Asignamos los nuevos valores encontrados al vector inicial.
    Y+= dt * (1/6.*K1 + 1/3. * K2 + 1/3.*K3 + 1/6.*K4)
    return(Y)

In [14]:
def visualgraph(Y,M,dt):
    cuerpos=[]
    history=np.array([Y])
    for i in range(0,len(Y),2):
        if (i <= 4):
            cuerpo = sphere(pos=Y[i],vel=Y[i+1],
                  mass=M[int(i/2)], color=color.yellow,
                  make_trail=True, interval=2, retain=1000,
                  radius = 0.5*M[int(i/2)]**(1.0/3.0))
        else:
            cuerpo = sphere(pos=Y[i],vel=Y[i+1],
                  mass=M[int(i/2)], color=color.red,
                  make_trail=True, interval=2, retain=1000,
                  radius = 0.5*M[int(i/2)]**(1.0/3.0))
        cuerpos.append(cuerpo)
    while(True):
        rate(100)
        history=np.append(history,[runge_kutta_orden4(dt,history[-1],M)],axis=0)
        for i in range(len(cuerpos)):
            cuerpos[i].pos = history[-1][int(i*2)]
            cuerpos[i].vel = history[-1][int((i*2)+1)]

In [6]:
#Calcula la velocidad del centro de masa del sistema.
def vcentre(Y,M):
    vcentre = 0
    mass = 0
    for i in range(0,len(Y),2):
        vcentre += Y[i+1]*M[int(i/2)]
        mass += M[int(i/2)]
    return (vcentre/mass)

In [7]:
#Definimos una función que llama varias veces el método de Runge-Kutta para efectuar la simulación.
def simulacion(M,Y,numIter=100,tmax=1,*args):
    dt = tmax/numIter #Se cuadra la escala de tiempo para que 1 periodo se ajuste al dt del método.
    #dt = 0.1
    history=np.array([Y]) #Este array contendrá la información luego de cada iteración con el fin de poder graficar.
    for iters in range(numIter):#Se ejecuta el método Runge-Kutta  durante la cantidad de iteraciones propuestas
        history=np.append(history,[runge_kutta_orden4(dt,history[-1],M)],axis=0)
    return (history)

In [8]:
#Definimos una función que grafica basados en los datos arrojados por la simulación
#Se pasa por parámetro la historia de la simulación y la trayectoria de la masa que se quiere graficar
#Si no se pasa ninguna masa por parámetro (o se iguala a 0) muestra la trayectoria de todas las masas
def graficador(history,masa=0,**args):
    colors = cm.rainbow(np.linspace(0, 1, len(history[0])/2))
    if (masa == 0):
        plt.figure(figsize=(10,10))
        #plt.axis('equal')
        for i in range(0,(int(len(history[0]))),2):
            plt.scatter(history[:,i][:,0],history[:,i][:,1],s=1)
            plt.scatter(history[:,i][:,0][0],history[:,i][:,1][0],s=50,label="Inicio",color=colors[int(i/2)])
        plt.legend(loc=0)
        plt.xlabel(u'Posición $x$')
        plt.ylabel(u'Posición $y$')
        plt.title(u'Trayectorias de las masas',fontsize=15)
        plt.show()
    else:
        plt.figure(figsize=(14,4))
        plt.axis('equal')
        plt.scatter(history[:,int((masa-1)*2)][:,0],history[:,int((masa-1)*2)][:,1],s=1)
        plt.scatter(history[:,int((masa-1)*2)][:,0][0],history[:,int((masa-1)*2)][:,1][0],s=50,label="Inicio",color='blue')
        plt.scatter(history[:,int((masa-1)*2)][:,0][-1],history[:,int((masa-1)*2)][:,1][-1],s=50,label="Fin",color='red')
        plt.legend(loc=0)
        plt.xlabel(u'Posición $x$')
        plt.ylabel(u'Posición $y$')
        plt.title(u'Trayectoria de $m_{%d}$'%masa,fontsize=15)
        plt.show()

In [9]:
#Animador
def animador(history):
    colors = cm.rainbow(np.linspace(0, 1, len(history[0])/2))
    for j in range(0,len(history),20):#Recorre toda la historia para graficar cada paso
        plt.figure(figsize=(9,7))
        plt.axis('equal')
        #plt.plot(history[:,0][:,0],history[:,0][:,1],'--',color='grey')
        #xlim = plt.gca().get_xlim()
        #ylim = plt.gca().get_ylim()
        plt.xlim(-20,20)#xlim)
        plt.ylim(-15,20)#ylim)
        for i in range(0,(int(len(history[0]))),2):#Grafica todas las masas por cada paso
            plt.scatter(history[:,i][:,0][j],history[:,i][:,1][j],s=50,color=colors[int(i/2)])
        plt.grid()
        plt.xlabel(u'Posición $x$')
        plt.ylabel(u'Posición $y$')
        plt.title('Gravitational Choreography',fontsize=15)
        plt.savefig('anim%d.png'%j)#Guarda las gráficas realizadas
        plt.cla()
        plt.close()
    os.system("convert -delay 12 -loop 0 $(ls anim*.png | sort -V) gravchorN%d.gif"%(int(len(history[0])/2)))
    os.system("rm anim*.png")#Toma las gráficas, las convierte en un .gif y las elimina

In [15]:
#Importación de datos
datos = np.genfromtxt('datos.csv',skip_header=1,delimiter=',')
InfoInicial = []
masas = []
for i in range(len(datos)):
    r = datos[i][2:5]
    v = datos[i][5:8]
    m = datos[i][1]
    InfoInicial.extend([r,v])
    masas.append(m)
#InfoInicial guarda la información de posiciones y velocidades iniciales
InfoInicial = np.array(InfoInicial)
#masas guarda la información de las masas
masas = np.array(masas)

In [16]:
#Resto la velocidad del centro de masa a todos los cuerpos
print"Velocidad inicial del centro de masa :",vcentre(InfoInicial,masas)
InfoInicial[1::2] -= vcentre(InfoInicial,masas)
print"Velocidad final del centro de masa :",vcentre(InfoInicial,masas)

Velocidad inicial del centro de masa : [ -2.74603539e-05   1.91775559e-04   0.00000000e+00]
Velocidad final del centro de masa : [  5.64404313e-20   1.00758217e-20   0.00000000e+00]


In [18]:
visualgraph(InfoInicial,masas,0.1)

TypeError: unsupported operand type(s) for ** or pow(): 'vector' and 'int'

In [ ]:
#Corremos la simulación para media unidad de tiempo y graficamos resultados.
hist=simulacion(Y=InfoInicial,M=masas,numIter=2000)
#graficador(hist,masa=1)
#graficador(hist,masa=2)
#graficador(hist,masa=3)
graficador(hist)